In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

 ## Section 01 : Setup

### 1.1: Install dependencies
The Kaggle Notebooks environment includes a pre-installed version of the [google-adk](https://google.github.io/adk-docs/) library for Python and its required dependencies.

To install and use ADK in your own Python development environment outside of this course, you can do so by running:

```
pip install google-adk

In [1]:
# pip install google-adk

### 1.2: Configure your Gemini API Key
This notebook uses the Gemini API, which requires an API key.

1. Get your API key

If you don't have one already, create an API key in Google AI Studio.

2. Add the key to Kaggle Secrets

Next, you will need to add your API key to your Kaggle Notebook as a Kaggle User Secret.

In the top menu bar of the notebook editor, select Add-ons then Secrets.
Create a new secret with the label GOOGLE_API_KEY.
Paste your API key into the "Value" field and click "Save".
Ensure that the checkbox next to GOOGLE_API_KEY is selected so that the secret is attached to the notebook.
3. Authenticate in the notebook

Run the cell below to access the GOOGLE_API_KEY you just saved and set it as an environment variable for the notebook to use:

In [104]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Gemini API key setup complete.


###  1.3: Import ADK components
Now, import the specific components you'll need from the Agent Development Kit and the Generative AI library. This keeps your code organized and ensures we have access to the necessary building blocks.

In [105]:
import os
import json

from google.genai import types  # For creating message Content/Parts
from typing import List, Dict, Any, Optional
from google.adk.agents import LlmAgent, Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService
from google.adk.tools import google_search, AgentTool, ToolContext
from google.adk.tools import load_memory, preload_memory
import asyncio
from google.adk.plugins.logging_plugin import LoggingPlugin

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


### 1.4: Helper functions
Helper function that prints the generated Python code and results from the code execution tool:

In [106]:
# Define helper functions that will be reused throughout the notebook
async def run_session(
    runner_instance: Runner,
    user_queries: list[str] | str = None, 
    session_name: str = "default",
):
    print(f"\n ### Session: {session_name}")

    # Get app name from the Runner
    app_name = runner_instance.app_name

    # Attempt to create a new session or retrieve an existing one
    try:
        session = await session_service.create_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )
    except:
        session = await session_service.get_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )

    # Process queries if provided
    if user_queries:
        # Convert single query to list for uniform processing
        if type(user_queries) == str:
            user_queries = [user_queries]

        # Process each query in the list sequentially
        for query in user_queries:
            print(f"\nUser > {query}")

            # Convert the query string to the ADK Content format
            query = types.Content(role="user", parts=[types.Part(text=query)])

            # Stream the agent's response asynchronously
            async for event in runner_instance.run_async(
                user_id=USER_ID, session_id=session.id, new_message=query
            ):
                # Check if the event contains valid content
                if event.content and event.content.parts:
                    # Filter out empty or "None" responses before printing
                    if (
                        event.content.parts[0].text != "None"
                        and event.content.parts[0].text
                    ):
                        print(f"{MODEL_NAME} > ", event.content.parts[0].text)
    else:
        print("No queries!")


print("✅ Helper functions defined.")

✅ Helper functions defined.


### 1.5: Configure Retry Options
When working with LLMs, you may encounter transient errors like rate limits or temporary service unavailability. Retry options automatically handle these failures by retrying the request with exponential backoff.

In [107]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

## 2.0 Section 02 : Multi Agent System

### 2.1 Creation of functions

In [108]:
def save_user_profile(
    tool_context: ToolContext,
    user_name: str,
    level: str,
    weakness: Optional[List[str]] = None
) -> Dict[str, Any]:
    """
    Saves the user's profile.
    
    Args:
        user_name: User's name
        level: Level (beginner, intermediate, advance)
        weakness: Weaknesses identified
    """
    tool_context.state["user:name"] = user_name
    tool_context.state["user:level"] = level
    tool_context.state["user:weakness"] = weakness or []
    
    return {
        "status": "success",
        "message": f"Profil's {user_name} recorded (level: {level})"
    }


In [109]:
def get_user_profile(tool_context: ToolContext) -> Dict[str, Any]:
    """
    Retrieves the user's profile recorded.
    """
    return {
        "name": tool_context.state.get("user:name", "Unknown"),
        "level": tool_context.state.get("user:level", "Not specified"),
        "weakness": tool_context.state.get("user:weakness", [])
    }

In [110]:
def save_quiz_result(
    tool_context: ToolContext,
    quiz_id: str,
    score: float,
    total_questions: int,
    correct_answers: int,
    topics_mastered: Optional[List[str]] = None,
    topics_to_review: Optional[List[str]] = None
) -> Dict[str, Any]:
    """
    Saves the quiz results for a progress tracking.
    
    Args:
        quiz_id: Unique id quiz
        score: Score as a pourcentage (0-100)
        total_questions: Total number of questions
        correct_answers: Total number of correct questions
        topics_mastered: Topics mastered
        topics_to_review: Topics to review
    """
    # Retrieve the history
    history = tool_context.state.get("quiz:history", [])
    
    # Add new result (data)
    result = {
        "quiz_id": quiz_id,
        "score": score,
        "total_questions": total_questions,
        "correct_answers": correct_answers,
        "topics_mastered": topics_mastered,
        "topics_to_review": topics_to_review
    }
    history.append(result)
    
    # Record
    tool_context.state["quiz:history"] = history
    tool_context.state["quiz:last_score"] = score
    
    # Update the weaknesses
    current_weakness = tool_context.state.get("user:weakness", [])
    if topics_to_review:
        updated_weakness = list(set(current_weakness + topics_to_review))
        tool_context.state["user:weakness"] = updated_weakness
    
    return {
        "status": "success",
        "message": f"Results: {score}% ({correct_answers}/{total_questions})",
        "progression": len(history)
    }



In [111]:
def get_learning_progression(tool_context: ToolContext) -> Dict[str, Any]:
    """
    Analyzes the user's progress based on their quiz history.
    """
    history = tool_context.state.get("quiz:history", [])
    
    if not history:
        return {
            "status": "no_data",
            "message": "No quizzes in progress"
        }
    
    # Calculation of statistics
    total_quizzes = len(history)
    avg_score = sum(q["score"] for q in history) / total_quizzes
    last_3_scores = [q["score"] for q in history[-3:]]
    
    # Trend Analysis
    if len(last_3_scores) >= 2:
        trend = "Improvement" if last_3_scores[-1] > last_3_scores[0] else "No change"
    else:
        trend = "Beginning"
    
    # Identifier les sujets récurrents à revoir
    all_weak_topics = []
    for quiz in history:
        all_weak_topics.extend(quiz.get("topics_to_review", []))
    
    from collections import Counter
    weak_topics_count = Counter(all_weak_topics)
    recurring_weaknesses = [topic for topic, count in weak_topics_count.items() if count >= 2]
    
    return {
        "total_quizzes": total_quizzes,
        "average_score": round(avg_score, 1),
        "last_score": history[-1]["score"],
        "trend": trend,
        "recurring_weaknesses": recurring_weaknesses,
        "last_3_scores": last_3_scores
    }

### 2.2 Creation of Agents

In [112]:
# Analyser Agent : Analyzes the content to identify the most relevant concepts for the future evaluation
# If the topic requires recent information : The agent use the tool "google_search" 
analyser_agent = LlmAgent(
    name="AnalyserAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="Analyzes the subject of study and identifies the key concepts to be evaluated.",
    instruction="""You are an expert in instructional analysis.
When given a topic for study:
1. Identify the 5-7 main concepts to master
2. Determine the necessary prerequisites
3. Suggest a logical learning progression
4. Evaluate the complexity of the topic

Format your response in a structured way with:
- Main concepts (list)
- Prerequisites (list)
- Suggested progression (numbered steps)
- Level of complexity (beginner/intermediate/advanced)
Use your existing knowledge to analyze the topic thoroughly.""",
   output_key="analysis_findings"  # The result of this agent will be stored in the session state with this key.
)

print("✅ analyser_agent.")

✅ analyser_agent.


In [113]:
# Question Agent : Generates questions and exercises adapted to the user's level
question_agent = LlmAgent(
    name="QuestionAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="Generates questions and exercises adapted to the user's level.",
    instruction="""You are an expert in creating educational quizzes.
Create a variety of questions:
1. True/False with justification
2. Single choice question (4 choices, 1 correct answer)
3. Multiple choice question (4 choices, 2 correct answers)
4. Simple practical exercises
5. Evaluate the complexity of the topic

Adapt the difficulty to the user's level:
- Beginner: basic comprehension questions
- Intermediate: application questions
- Advanced: analysis and synthesis questions

For each question, provide:
- A clear statement
- The options (if multiple choice)
- The correct answer
- A brief explanation
- The concept being tested
- The difficulty level (1-5).""",
    
    output_key="generated_questions"  # The result of this agent will be stored in the session state with this key.
)

print("✅ question_agent.")

✅ question_agent.


In [114]:
# Explanation Agent : Generates explanation to the user
explanation_agent = LlmAgent(
    name="ExplanationAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="Generates detailed and educational explanations.",
    instruction="""You are an expert in pedagogy.
Generates explanations that:
1. Are adapted to the user's level.
2. Use concrete examples.
3. Offer relevant analogies.
4. Break down complex concepts in a simple way.
5. Include mnemonic devices if useful.

Adapt the difficulty to the user's level:
- Beginner: basic comprehension questions.
- Intermediate: application questions.
- Advanced: analysis and synthesis questions.

For each question, provide:
- A clear statement.
- The options (if multiple choice).
- The correct answer.
- A brief explanation.
- The concept being tested.
- The difficulty level (1-5).

Keep your tone encouraging and positive.""",
    
    output_key="generated_explanation"  # The result of this agent will be stored in the session state with this key.
)

print("✅ explanation_agent.")

✅ explanation_agent.


In [115]:
# Feedback Agent : Coach the user with personalized and constructive feedback
feedback_agent = LlmAgent(
    name="FeedbackAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="Generates personalized and constructive feedback.",
    instruction="""You are a supportive learning coach.
Analyze the user's performance and provide:
1. Positive feedback on successes.
2. Constructive analysis of errors.
3. Targeted improvement tips.
4. Revision recommendations.
5. An action plan for the next steps.

Your feedback should be:
- Encouraging and motivating.
- Positive and constructive.
- Specific and actionable.
- Adapted to the user's level.
- Progress-oriented.

Highlight strengths before areas for improvement.""",
    
    output_key="generated_feedback"  # The result of this agent will be stored in the session state with this key.
)

print("✅ feedback_agent.")

✅ feedback_agent.


In [116]:
# Root Coordinator: Orchestrates the workflow by calling the sub-agents as tools.
root_agent = LlmAgent(
    name="QuizAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="Main coordinator.",
    instruction="""You are the main coordinator of the quiz generation system..
 You must follow these steps:
1. Start by saving the user's profile with `save_user_profile` if new information is provided.
2. Use `analyser_agent` to analyze the requested topic.
3. Use `question_agent` to create appropriate questions.
4. Use `explanation_agent` for additional explanations.
5. After a quiz, use `feedback_agent` for personalized feedback.
6. Save the results with `save_quiz_result`.
7. Consult `get_learning_progression` to adjust the difficulty.
8. Use `get_user_profile` to personalize the experience.


Be proactive in personalizing and tracking progress.""",
    
   tools=[
        save_user_profile,
        get_user_profile,
        save_quiz_result,
        get_learning_progression
       
    ],
    sub_agents=[
        analyser_agent,
        question_agent,
        explanation_agent,
        feedback_agent
    ]
)

print("✅ root_agent created.")

✅ root_agent created.


In [117]:
class QuizGeneratorSystem:
    """Quiz generation system"""
    
    def __init__(self):
        self.app_name = "quiz_generator_app"
        self.user_id = "user_01"
        
        # Services
        self.session_service = InMemorySessionService()
        self.memory_service = InMemoryMemoryService()
        
        # Runner avec plugin de logging
        self.runner = Runner(
            agent=root_agent,
            app_name=self.app_name,
            session_service=self.session_service,
            memory_service=self.memory_service,
            plugins=[LoggingPlugin()]
        )
    
    async def create_session(self, session_id: str = None):
        """Create or retrieve a session"""
        if not session_id:
            import uuid
            session_id = f"session_{uuid.uuid4().hex[:8]}"
        
        try:
            session = await self.session_service.create_session(
                app_name=self.app_name,
                user_id=self.user_id,
                session_id=session_id
            )
        except:
            session = await self.session_service.get_session(
                app_name=self.app_name,
                user_id=self.user_id,
                session_id=session_id
            )
        
        return session
    
    async def run(self, user_query: str, session_id: str = None):
        """
        User request
        
        Args:
            user_query: User's question
            session_id: Id session (will be creade if none)
        """
        # Create or retrieve the session
        session = await self.create_session(session_id)
        
        # User's query
        query_content = types.Content(
            role="user",
            parts=[types.Part(text=user_query)]
        )
        
        print(f"\n{'='*60}")
        print(f"User > {user_query}")
        print(f"{'='*60}\n")
        
        # Run and display the responses
        async for event in self.runner.run_async(
            user_id=self.user_id,
            session_id=session.id,
            new_message=query_content
        ):
            if event.is_final_response() and event.content and event.content.parts:
                for part in event.content.parts:
                    if hasattr(part, 'text') and part.text and part.text != "None":
                        print(f"Assistant > {part.text}\n")
        
        print(f"{'='*60}\n")
        
        # Save the session to memory
        await self.memory_service.add_session_to_memory(session)
        
        return session.id


In [118]:
async def demo_simple():
    """Example of use"""
    
    system = QuizGeneratorSystem()
    
    await system.run(
        "Je suis Paul, niveau lycée. Crée-moi un quiz sur les fonctions exponentielles (5 questions)."
    )

In [119]:
if __name__ == "__main__":
    import asyncio  
    
   
    choice = input("\nType 1 to start the session: ").strip()
    
    if choice == "1":
       if choice == "1":
        await demo_simple()
    else:
        print("\nType 1 to start the session, please")


Type 1 to start the session:  1



User > Je suis Paul, niveau lycée. Crée-moi un quiz sur les fonctions exponentielles (5 questions).

[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-9bf8b78b-8e9f-47f0-9fd3-eb5a9789a91b
[logging_plugin]    Session ID: session_f299fffc
[logging_plugin]    User ID: user_01
[logging_plugin]    App Name: quiz_generator_app
[logging_plugin]    Root Agent: QuizAgent
[logging_plugin]    User Content: text: 'Je suis Paul, niveau lycée. Crée-moi un quiz sur les fonctions exponentielles (5 questions).'
[logging_plugin] 🏃 INVOCATION STARTING
[logging_plugin]    Invocation ID: e-9bf8b78b-8e9f-47f0-9fd3-eb5a9789a91b
[logging_plugin]    Starting Agent: QuizAgent
[logging_plugin] 🤖 AGENT STARTING
[logging_plugin]    Agent Name: QuizAgent
[logging_plugin]    Invocation ID: e-9bf8b78b-8e9f-47f0-9fd3-eb5a9789a91b
[logging_plugin] 🧠 LLM REQUEST
[logging_plugin]    Model: gemini-2.5-flash-lite
[logging_plugin]    Agent: QuizAgent
[logging_plugin]    System Instruction: 'You 

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: QuizAgent
[logging_plugin]    Content: function_call: save_user_profile | function_call: transfer_to_agent
[logging_plugin]    Token Usage - Input: 912, Output: 45
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: c8fd7f85-9ef9-43c0-b9d4-d19ea4754a6e
[logging_plugin]    Author: QuizAgent
[logging_plugin]    Content: function_call: save_user_profile | function_call: transfer_to_agent
[logging_plugin]    Final Response: False
[logging_plugin]    Function Calls: ['save_user_profile', 'transfer_to_agent']
[logging_plugin] 🔧 TOOL STARTING
[logging_plugin]    Tool Name: save_user_profile
[logging_plugin]    Agent: QuizAgent
[logging_plugin]    Function Call ID: adk-ce7cf9a8-8166-4bec-9bd3-a5d3b20f77f0
[logging_plugin]    Arguments: {'user_name': 'Paul', 'level': 'lycée'}
[logging_plugin] 🔧 TOOL COMPLETED
[logging_plugin]    Tool Name: save_user_profile
[logging_plugin]    Agent: QuizAgent
[logging_plugin]    Function Cal

In [120]:
# Méthode 1 - Simple (avec vérification)
result = await demo_simple()
if result and result['quiz_content']:
    print(result['quiz_content'])
else:
    print("Aucun quiz généré")


User > Je suis Paul, niveau lycée. Crée-moi un quiz sur les fonctions exponentielles (5 questions).

[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-b2650686-5690-45a3-a22b-416b3241a363
[logging_plugin]    Session ID: session_abd729b6
[logging_plugin]    User ID: user_01
[logging_plugin]    App Name: quiz_generator_app
[logging_plugin]    Root Agent: QuizAgent
[logging_plugin]    User Content: text: 'Je suis Paul, niveau lycée. Crée-moi un quiz sur les fonctions exponentielles (5 questions).'
[logging_plugin] 🏃 INVOCATION STARTING
[logging_plugin]    Invocation ID: e-b2650686-5690-45a3-a22b-416b3241a363
[logging_plugin]    Starting Agent: QuizAgent
[logging_plugin] 🤖 AGENT STARTING
[logging_plugin]    Agent Name: QuizAgent
[logging_plugin]    Invocation ID: e-b2650686-5690-45a3-a22b-416b3241a363
[logging_plugin] 🧠 LLM REQUEST
[logging_plugin]    Model: gemini-2.5-flash-lite
[logging_plugin]    Agent: QuizAgent
[logging_plugin]    System Instruction: 'You 

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: QuizAgent
[logging_plugin]    Content: function_call: save_user_profile | function_call: transfer_to_agent
[logging_plugin]    Token Usage - Input: 912, Output: 45
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: 1feaf689-7762-4c0b-85da-c6a1c088e5b3
[logging_plugin]    Author: QuizAgent
[logging_plugin]    Content: function_call: save_user_profile | function_call: transfer_to_agent
[logging_plugin]    Final Response: False
[logging_plugin]    Function Calls: ['save_user_profile', 'transfer_to_agent']
[logging_plugin] 🔧 TOOL STARTING
[logging_plugin]    Tool Name: save_user_profile
[logging_plugin]    Agent: QuizAgent
[logging_plugin]    Function Call ID: adk-53a6e444-92f2-4d1a-b76e-615c3067ea56
[logging_plugin]    Arguments: {'user_name': 'Paul', 'level': 'lycée'}
[logging_plugin] 🔧 TOOL COMPLETED
[logging_plugin]    Tool Name: save_user_profile
[logging_plugin]    Agent: QuizAgent
[logging_plugin]    Function Cal

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: AnalyserAgent
[logging_plugin]    Content: function_call: transfer_to_agent
[logging_plugin]    Token Usage - Input: 592, Output: 20
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: aa89740f-c3b5-4f37-a7ac-1e4655458af6
[logging_plugin]    Author: AnalyserAgent
[logging_plugin]    Content: function_call: transfer_to_agent
[logging_plugin]    Final Response: False
[logging_plugin]    Function Calls: ['transfer_to_agent']
[logging_plugin] 🔧 TOOL STARTING
[logging_plugin]    Tool Name: transfer_to_agent
[logging_plugin]    Agent: AnalyserAgent
[logging_plugin]    Function Call ID: adk-891840ed-1d0c-4991-ac31-d62b202af2c6
[logging_plugin]    Arguments: {'agent_name': 'ExplanationAgent'}
[logging_plugin] 🔧 TOOL COMPLETED
[logging_plugin]    Tool Name: transfer_to_agent
[logging_plugin]    Agent: AnalyserAgent
[logging_plugin]    Function Call ID: adk-891840ed-1d0c-4991-ac31-d62b202af2c6
[logging_plugin]    Result: {}
[l

## Debug

In [123]:
# VERSION CORRIGÉE AVEC DIAGNOSTIC
# Cette version capture TOUS les événements pour identifier le problème

from google.genai import types
from typing import List, Dict, Any, Optional
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService
from google.adk.tools import google_search, ToolContext
import json

# Configuration retry
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)

# Fonctions de gestion (simplifiées pour éviter les erreurs)
def save_user_profile(
    tool_context: ToolContext,
    user_name: str,
    level: str,
    weakness: Optional[List[str]] = None
) -> Dict[str, Any]:
    """Sauvegarde le profil utilisateur"""
    tool_context.state["user:name"] = user_name
    tool_context.state["user:level"] = level
    tool_context.state["user:weakness"] = weakness or []
    
    return {
        "status": "success",
        "message": f"Profil de {user_name} enregistré (niveau: {level})"
    }

def get_user_profile(tool_context: ToolContext) -> Dict[str, Any]:
    """Récupère le profil utilisateur"""
    return {
        "name": tool_context.state.get("user:name", "Inconnu"),
        "level": tool_context.state.get("user:level", "Non spécifié"),
        "weakness": tool_context.state.get("user:weakness", [])
    }

# Agent principal SIMPLIFIÉ pour éviter les conflits
main_agent = LlmAgent(
    name="QuizGeneratorAgent",
    model=Gemini(
        model="gemini-2.0-flash-exp",
        retry_options=retry_config
    ),
    description="Système de génération de quiz intelligent.",
    instruction="""Tu es un système de génération de quiz éducatif.

TÂCHE:
Quand un utilisateur demande un quiz:
1. Enregistre son profil avec save_user_profile(user_name, level)
2. Génère immédiatement le quiz adapté à son niveau
3. Formate le quiz de manière claire et structurée

FORMAT DU QUIZ:
```
📚 QUIZ - [Sujet]
Niveau: [niveau de l'utilisateur]

### Question 1
[Texte de la question]

A) [Option A]
B) [Option B]
C) [Option C]
D) [Option D]

### Question 2
[...]
```

IMPORTANT: 
- Génère le quiz DIRECTEMENT sans attendre
- Sois clair et structuré
- Adapte la difficulté au niveau (lycée = niveau intermédiaire)
""",
    
    tools=[
        save_user_profile,
        get_user_profile
    ]
)

# Classe système avec DIAGNOSTIC COMPLET
class QuizGeneratorSystem:
    """Quiz generation system with full diagnostic"""
    
    def __init__(self, debug=True):
        self.app_name = "quiz_generator_app"
        self.user_id = "user_01"
        self.debug = debug
        
        self.session_service = InMemorySessionService()
        self.memory_service = InMemoryMemoryService()
        
        self.runner = Runner(
            agent=main_agent,
            app_name=self.app_name,
            session_service=self.session_service,
            memory_service=self.memory_service
        )
        
        self.last_response = ""
        self.all_events = []  # Pour le diagnostic
    
    async def create_session(self, session_id: str = None):
        if not session_id:
            import uuid
            session_id = f"session_{uuid.uuid4().hex[:8]}"
        
        try:
            session = await self.session_service.create_session(
                app_name=self.app_name,
                user_id=self.user_id,
                session_id=session_id
            )
        except:
            session = await self.session_service.get_session(
                app_name=self.app_name,
                user_id=self.user_id,
                session_id=session_id
            )
        
        return session
    
    async def run(self, user_query: str, session_id: str = None, display: bool = True):
        """
        Execute query with full diagnostic
        """
        session = await self.create_session(session_id)
        
        query_content = types.Content(
            role="user",
            parts=[types.Part(text=user_query)]
        )
        
        if display:
            print(f"\n{'='*70}")
            print(f"👤 User > {user_query}")
            print(f"{'='*70}\n")
        
        # Compteurs pour diagnostic
        event_count = 0
        final_response_count = 0
        text_parts_count = 0
        full_response = []
        
        try:
            async for event in self.runner.run_async(
                user_id=self.user_id,
                session_id=session.id,
                new_message=query_content
            ):
                event_count += 1
                self.all_events.append(event)
                
                if self.debug:
                    print(f"🔍 Event {event_count}: ", end="")
                    print(f"final={event.is_final_response()}, ", end="")
                    print(f"has_content={event.content is not None}, ", end="")
                    if event.content:
                        print(f"parts={len(event.content.parts) if event.content.parts else 0}")
                    else:
                        print("no_content")
                
                # Capturer TOUS les événements avec du texte
                if event.content and event.content.parts:
                    for part in event.content.parts:
                        if hasattr(part, 'text') and part.text:
                            text_parts_count += 1
                            if part.text != "None" and part.text.strip():
                                full_response.append(part.text)
                                if display:
                                    print(f"🤖 Assistant > {part.text}\n")
                
                # Compter les réponses finales
                if event.is_final_response():
                    final_response_count += 1
        
        except Exception as e:
            print(f"❌ Erreur pendant l'exécution: {e}")
            import traceback
            traceback.print_exc()
        
        # Diagnostic
        if self.debug:
            print(f"\n📊 DIAGNOSTIC:")
            print(f"   - Total événements: {event_count}")
            print(f"   - Réponses finales: {final_response_count}")
            print(f"   - Parties texte: {text_parts_count}")
            print(f"   - Contenu capturé: {len(full_response)} parties")
        
        if display:
            print(f"{'='*70}\n")
        
        # Stocker la réponse complète
        self.last_response = "\n".join(full_response)
        
        await self.memory_service.add_session_to_memory(session)
        
        # Vérifier si on a du contenu
        if not self.last_response.strip():
            print("⚠️ ATTENTION: Aucun contenu textuel capturé!")
            print("💡 Suggestions:")
            print("   1. Vérifiez que votre clé API Gemini est valide")
            print("   2. Vérifiez votre quota API")
            print("   3. Essayez avec gemini-1.5-flash ou gemini-1.5-pro")
            return None
        
        return {
            "session_id": session.id,
            "quiz_content": self.last_response,
            "events_count": event_count,
            "final_responses": final_response_count
        }
    
    def get_last_quiz(self):
        """Retourne le dernier quiz généré"""
        return self.last_response
    
    def save_quiz_to_file(self, filename: str = "quiz.txt"):
        """Sauvegarde le quiz dans un fichier"""
        if not self.last_response.strip():
            print("⚠️ Aucun contenu à sauvegarder")
            return False
        
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(self.last_response)
        print(f"✅ Quiz sauvegardé dans {filename}")
        return True
    
    def get_diagnostic(self):
        """Affiche un diagnostic complet"""
        print(f"\n🔍 DIAGNOSTIC COMPLET:")
        print(f"   - Événements capturés: {len(self.all_events)}")
        print(f"   - Contenu stocké: {len(self.last_response)} caractères")
        print(f"   - Contenu vide: {not self.last_response.strip()}")
        
        if self.all_events:
            print(f"\n   Types d'événements:")
            for i, event in enumerate(self.all_events[:5]):  # Premiers 5
                print(f"      {i+1}. Final: {event.is_final_response()}, Content: {event.content is not None}")

# VERSION ALTERNATIVE avec modèle différent
main_agent_alternative = LlmAgent(
    name="QuizGeneratorAgentAlt",
    model=Gemini(
        model="gemini-1.5-flash",  # Modèle alternatif plus stable
        retry_options=retry_config
    ),
    description="Système de génération de quiz (version alternative).",
    instruction="""Tu es un générateur de quiz éducatif.

Génère immédiatement un quiz au format:

📚 QUIZ - [Sujet]

### Question 1
[Question]
A) [Option]
B) [Option]
C) [Option]
D) [Option]

Génère toutes les questions demandées.""",
    
    tools=[save_user_profile, get_user_profile]
)

class QuizGeneratorSystemAlt(QuizGeneratorSystem):
    """Version alternative avec gemini-1.5-flash"""
    def __init__(self, debug=True):
        super().__init__(debug)
        self.runner = Runner(
            agent=main_agent_alternative,
            app_name=self.app_name,
            session_service=self.session_service,
            memory_service=self.memory_service
        )

# Fonctions de test
async def test_simple():
    """Test simple avec diagnostic"""
    print("🧪 TEST SIMPLE avec gemini-2.0-flash-exp\n")
    
    system = QuizGeneratorSystem(debug=True)
    
    result = await system.run(
        "Je suis Paul, niveau lycée. Crée un quiz de 3 questions sur les fonctions exponentielles."
    )
    
    if result and result.get('quiz_content'):
        print("\n✅ SUCCÈS!")
        print(f"📝 Quiz généré: {len(result['quiz_content'])} caractères")
        system.save_quiz_to_file("test_quiz.txt")
        return system, result
    else:
        print("\n❌ ÉCHEC - Aucun contenu")
        system.get_diagnostic()
        return None, None

async def test_alternative():
    """Test avec modèle alternatif"""
    print("🧪 TEST ALTERNATIF avec gemini-1.5-flash\n")
    
    system = QuizGeneratorSystemAlt(debug=True)
    
    result = await system.run(
        "Je suis Marie, niveau lycée. Crée un quiz de 3 questions sur les dérivées."
    )
    
    if result and result.get('quiz_content'):
        print("\n✅ SUCCÈS!")
        print(f"📝 Quiz généré: {len(result['quiz_content'])} caractères")
        system.save_quiz_to_file("test_quiz_alt.txt")
        return system, result
    else:
        print("\n❌ ÉCHEC - Aucun contenu")
        system.get_diagnostic()
        return None, None

async def test_minimal():
    """Test minimal sans outils"""
    print("🧪 TEST MINIMAL (sans outils)\n")
    
    # Agent super simplifié
    minimal_agent = LlmAgent(
        name="MinimalAgent",
        model=Gemini(model="gemini-1.5-flash"),
        description="Agent minimal",
        instruction="Génère un quiz de 3 questions sur le sujet demandé. Format simple avec Question 1, Question 2, Question 3."
    )
    
    system = QuizGeneratorSystem(debug=True)
    system.runner = Runner(
        agent=minimal_agent,
        app_name=system.app_name,
        session_service=system.session_service,
        memory_service=system.memory_service
    )
    
    result = await system.run(
        "Crée un quiz de 3 questions sur les mathématiques."
    )
    
    if result:
        print("\n✅ Test minimal réussi!")
        return system, result
    else:
        print("\n❌ Même le test minimal échoue - problème API")
        return None, None

# Instructions
print("""
✅ SYSTÈME DE DIAGNOSTIC PRÊT!

Exécutez dans l'ordre:

# 1. TEST SIMPLE (recommandé en premier)
system, result = await test_simple()

# 2. Si échec, TEST ALTERNATIF
system, result = await test_alternative()

# 3. Si encore échec, TEST MINIMAL
system, result = await test_minimal()

# 4. Si succès, utilisez normalement:
if system and result:
    print(result['quiz_content'])
    system.save_quiz_to_file("mon_quiz.txt")
""")


✅ SYSTÈME DE DIAGNOSTIC PRÊT!

Exécutez dans l'ordre:

# 1. TEST SIMPLE (recommandé en premier)
system, result = await test_simple()

# 2. Si échec, TEST ALTERNATIF
system, result = await test_alternative()

# 3. Si encore échec, TEST MINIMAL
system, result = await test_minimal()

# 4. Si succès, utilisez normalement:
if system and result:
    print(result['quiz_content'])
    system.save_quiz_to_file("mon_quiz.txt")



In [125]:
system, result = await test_simple()

🧪 TEST SIMPLE avec gemini-2.0-flash-exp


👤 User > Je suis Paul, niveau lycée. Crée un quiz de 3 questions sur les fonctions exponentielles.

❌ Erreur pendant l'exécution: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash-exp\n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 0, model: gemini-2.0-flash-exp\nPlease retry in 42.511182673s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-ap

Traceback (most recent call last):
  File "/tmp/ipykernel_48/3749166811.py", line 157, in run
    async for event in self.runner.run_async(
  File "/usr/local/lib/python3.11/dist-packages/google/adk/runners.py", line 443, in run_async
    async for event in agen:
  File "/usr/local/lib/python3.11/dist-packages/google/adk/runners.py", line 427, in _run_with_trace
    async for event in agen:
  File "/usr/local/lib/python3.11/dist-packages/google/adk/runners.py", line 653, in _exec_with_plugin
    async for event in agen:
  File "/usr/local/lib/python3.11/dist-packages/google/adk/runners.py", line 416, in execute
    async for event in agen:
  File "/usr/local/lib/python3.11/dist-packages/google/adk/agents/base_agent.py", line 294, in run_async
    async for event in agen:
  File "/usr/local/lib/python3.11/dist-packages/google/adk/agents/llm_agent.py", line 435, in _run_async_impl
    async for event in agen:
  File "/usr/local/lib/python3.11/dist-packages/google/adk/flows/llm_flows/base

In [97]:
async def check_quota():
    """Vérifie si l'API est accessible"""
    print("🔍 VÉRIFICATION DE L'API...\n")
    
    try:
        import google.generativeai as genai
        import os
        
        api_key = os.environ.get("GOOGLE_API_KEY")
        genai.configure(api_key=api_key)
        
        model = genai.GenerativeModel('gemini-1.5-flash-8b')
        response = model.generate_content("Dis juste 'OK'")
        
        print("✅ API accessible!")
        print(f"Réponse: {response.text}\n")
        return True
        
    except Exception as e:
        if "429" in str(e):
            print("❌ Quota épuisé (429)")
            print("\n💡 SOLUTIONS:")
            print("   1. ⏰ Attendez 1-2 minutes")
            print("   2. 🔑 Vérifiez sur: https://aistudio.google.com/apikey")
            print("   3. 📊 Limites gratuites: 15 req/min, 1500 req/jour")
            print("   4. 🆕 Créez une nouvelle clé API si nécessaire")
        else:
            print(f"❌ Erreur: {e}")
        return False

# Instructions
print("""
✅ SYSTÈME ANTI-QUOTA PRÊT!

ÉTAPE 1 - Vérifier l'API:
quota_ok = await check_quota()

ÉTAPE 2 - Si quota OK, test optimisé:
system, result = await test_optimized()

ÉTAPE 3 - Si encore erreur 429, méthode directe:
quiz = await test_direct()

ÉTAPE 4 - Si tout échoue:
⏰ Attendez 1-2 minutes puis recommencez

💡 ASTUCE: Entre chaque test, attendez 10-15 secondes!
""")


✅ SYSTÈME ANTI-QUOTA PRÊT!

ÉTAPE 1 - Vérifier l'API:
quota_ok = await check_quota()

ÉTAPE 2 - Si quota OK, test optimisé:
system, result = await test_optimized()

ÉTAPE 3 - Si encore erreur 429, méthode directe:
quiz = await test_direct()

ÉTAPE 4 - Si tout échoue:
⏰ Attendez 1-2 minutes puis recommencez

💡 ASTUCE: Entre chaque test, attendez 10-15 secondes!

